In [2]:
from dataset import *
from torch import nn
from torch import optim
from train_loop import *
import csv
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, Dataset, random_split, ConcatDataset, Subset
from tsai.models.InceptionTimePlus import InceptionTimePlus
import gc
import json
import sys

In [3]:
set_seed()

In [4]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [5]:
mimic_ds = MimicDataset(CSV_FILE, ROOT_DIR)

False


In [7]:
muse_ds = MuseDataset(MUSE_XPATH_DIV5, MUSE_YPATH, divide_by_5=False)

In [8]:
muse_ds[0]

(tensor([[-0.1260, -0.0970, -0.0530,  ..., -0.0530, -0.0040, -0.0090],
         [-0.2340, -0.2240, -0.1850,  ...,  0.0090,  0.0040,  0.0090],
         [-0.1070, -0.1260, -0.1310,  ...,  0.0630,  0.0090,  0.0190],
         ...,
         [-0.2530, -0.2140, -0.1220,  ..., -0.0730, -0.0780, -0.0630],
         [-0.1950, -0.1650, -0.1170,  ..., -0.0730, -0.0780, -0.0630],
         [-0.1260, -0.1170, -0.1070,  ..., -0.0680, -0.0630, -0.0730]]),
 tensor([461.]))

In [9]:
muse_ds_bias_fixed = MuseDataset(MUSE_XPATH_DIV5, MUSE_YPATH, divide_by_5=False, correct_bias=True)

In [10]:
auto_qtcms_ds = HandDataset(X_HAND_PATH, Y_HAND_PATH, qt_type="auto")
auto_qtcms_loader = DataLoader(auto_qtcms_ds, batch_size=BATCH_SIZE, shuffle=False)

In [11]:
expert_qtcms_ds = HandDataset(X_HAND_PATH, Y_HAND_PATH, qt_type="expert")
expert_qtcms_loader = DataLoader(expert_qtcms_ds, batch_size=BATCH_SIZE, shuffle=False)

In [12]:
rdvq_ds = ECGRDVQDataset(CSV_FILE_RDVQ, ROOT_DIR_RDVQ)
val_loader_rdvq = DataLoader(rdvq_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

False


In [15]:
# we cannot publish this type of data as it contains specific server details.
with open("mimic_muse_slice_val/indices.json") as json_file:
    data = json.load(json_file)

mimic_untouched = list(map(int, data["mimic"].keys()))
muse_untouched = data["muse"]
ptb_untouched = list(map(int, data["ptb"].keys()))

In [16]:
val_indices_mimic = mimic_untouched
val_indices_muse = muse_untouched

val_subset_mimic = Subset(mimic_ds, val_indices_mimic)
val_subset_muse = Subset(muse_ds, val_indices_muse)
val_subset_muse_bias_fixed = Subset(muse_ds_bias_fixed, val_indices_muse)

val_loader_mimic = DataLoader(val_subset_mimic, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
val_loader_muse = DataLoader(val_subset_muse, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
val_loader_muse_bias_fixed = DataLoader(val_subset_muse_bias_fixed, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)


In [17]:
ptbptb_full_ds = PTBDataSet(PTB_PRECOMP_DF_PATH, PTB_DATASET_PATH, cache_rr=False, do_sanity_check=True)
subset_ptb = Subset(ptbptb_full_ds, ptb_untouched)
ptb_loader = DataLoader(subset_ptb, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

In [18]:
mimic_slices = [60_150]
muse_slices = [60_150]
lano_bias_correction = [True]

In [19]:
for m in zip(mimic_slices, muse_slices, lano_bias_correction):
    print(m[0], m[1], m[2])

-1 -1 True


In [20]:
original_stdout = sys.stdout

In [21]:
indices = {"mimic": {mimic_slice: None for mimic_slice in mimic_slices}, 
           "muse": {muse_slice: None for muse_slice in muse_slices}}
indices

{'mimic': {-1: None}, 'muse': {-1: None}}

In [25]:
for mimic_set, muse_set, bias_correcton in zip(mimic_slices, muse_slices, lano_bias_correction):
    criterion = nn.L1Loss()  # mae loss
    model = InceptionTimePlus(c_in=12, c_out=1).to(device)
    optimizer = optim.Adam(model.parameters(), lr=INIT_LR)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", patience=3, factor=0.1, min_lr=1e-6
    )

    print(f"Mimic: {mimic_set}, muse {muse_set}, bias {bias_correcton}")

    combined_dataset, mimic_indices, muse_indices = create_combined_dataset(
        CSV_FILE,
        ROOT_DIR,
        MUSE_XPATH_DIV5,
        MUSE_YPATH,
        mimic_samples=mimic_set,
        muse_samples=muse_set,
        return_indices=True,
        mimic_untouched=mimic_untouched,
        muse_untouched=muse_untouched,
        correct_bias=bias_correcton
    )

    dataset_size = len(combined_dataset)
    test_size = int(TEST_SPLIT * dataset_size)
    train_size = dataset_size - test_size

    train_dataset, test_dataset = random_split(
        combined_dataset, [train_size, test_size]
    )

    train_loader = DataLoader(
        train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS
    )
    test_loader = DataLoader(
        test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    tensorboard_writer = SummaryWriter(log_dir=f"mimic_muse_slice_val/mimic_{mimic_set}_muse_{muse_set}_bias_{bias_correcton}")

    with open(f"mimic_muse_slice_val/logs/mimic_{mimic_set}_muse_{muse_set}_bias_{bias_correcton}.log", "a+") as log_file:
        sys.stdout = Tee(original_stdout, log_file)  # Set Tee to log to both

        # Logging both to stdout and file
        print(f"Starting training with slice size: Mimic: {mimic_set}, muse {muse_set} bias_{bias_correcton}")

        print(f"Subset created with {len(combined_dataset)} samples.")
        print(f"DS size: {dataset_size}, train: {train_size}, test: {test_size}")

        train_model(
            model,
            train_loader,
            test_loader,
            criterion,
            optimizer,
            scheduler,
            tensorboard_writer,
            100,
            device,
            debug=True,
            early_stopper_patience=5,
            early_stopper_min_delta=0.01
        )

        # mae, mse, rsme = check_accuracy(model, hand_loader, device)
        print("Mimic")
        mimic_acc = check_accuracy(model, val_loader_mimic, device)
        
        print("Muse biased")
        muse_acc = check_accuracy(model, val_loader_muse, device)

        print("Muse unbiased")
        muse_acc_bias_fixed = check_accuracy(model, val_loader_muse_bias_fixed, device)
        
        print("QTcMS auto")
        hand_acc_auto = check_accuracy(model, auto_qtcms_loader, device)

        print("QTcMS expert")
        hand_acc_expert = check_accuracy(model, expert_qtcms_loader, device)
        
        print("PTB")
        ptb_acc = check_accuracy(model, ptb_loader, device)

        print("ECGRDVQ")
        rdvq_acc = check_accuracy(model, val_loader_rdvq, device)

        csv_path = "mimic_muse_slice_val/logs/mimic+muse.csv"
        fieldnames = [
            "size_mimic", "size_muse", "bias_corrected", "mae_mimic", "mse_mimic", "rmse_mimic",
            "mae_muse", "mse_muse", "rmse_muse",
            "mae_muse_bias_fixed", "mse_muse_bias_fixed", "rmse_muse_bias_fixed",
            "mae_hand_auto", "mse_hand_auto", "rmse_hand_auto",
            "mae_hand_expert", "mse_hand_expert", "rmse_hand_expert",
            "mae_ptb", "mse_ptb", "rmse_ptb",
            "mae_rdvq", "mse_rdvq", "rmse_rdvq"
        ]

        try:
            with open(csv_path, "r") as f:
                is_file_empty = f.read().strip() == ""
        except FileNotFoundError:
            is_file_empty = True

        # Append data to the file
        with open(csv_path, "a", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)

            # Write header if file is empty
            if is_file_empty:
                writer.writeheader()
            data = {
                "size_mimic": mimic_set, "size_muse": muse_set, "bias_corrected": bias_correcton,
                "mae_mimic": mimic_acc[0], "mse_mimic": mimic_acc[1], "rmse_mimic": mimic_acc[2],
                "mae_muse": muse_acc[0], "mse_muse": muse_acc[1], "rmse_muse": muse_acc[2],
                "mae_muse_bias_fixed": muse_acc_bias_fixed[0], "mse_muse_bias_fixed": muse_acc_bias_fixed[1], "rmse_muse_bias_fixed": muse_acc_bias_fixed[2],
                "mae_hand_auto": hand_acc_auto[0], "mse_hand_auto": hand_acc_auto[1], "rmse_hand_auto": hand_acc_auto[2],
                "mae_hand_expert": hand_acc_expert[0], "mse_hand_expert": hand_acc_expert[1], "rmse_hand_expert": hand_acc_expert[2],
                "mae_ptb": ptb_acc[0], "mse_ptb": ptb_acc[1], "rmse_ptb": ptb_acc[2],
                "mae_rdvq": rdvq_acc[0], "mse_rdvq": rdvq_acc[1], "rmse_rdvq": rdvq_acc[2]
            }
            writer.writerow(data)

        # Simulated training step (replace with actual code)
        print("Training completed.")
        sys.stdout = original_stdout
    tensorboard_writer.close()
    print("cleaning cache")
    # Restore original stdout
    sys.stdout = original_stdout
    del (
        criterion,
        model,
        optimizer,
        scheduler,
        train_dataset,
        test_dataset,
        train_loader,
        test_loader,
    )
    torch.cuda.empty_cache()
    print(f"Finished Slice: Mimic: {mimic_set}, muse {muse_set}")
    gc.collect()

Mimic: -1, muse -1, bias True
False
Starting training with slice size: Mimic: -1, muse -1 bias_True
Subset created with 625951 samples.
DS size: 625951, train: 500761, test: 125190


Epoch 1 - Training: 100%|██████████| 3913/3913 [03:45<00:00, 17.38it/s]


Model checkpoint epoch 0 saved to mimic_muse_slice_val/mimic_-1_muse_-1_bias_True/best_model.pth.tar
Epoch [1/100] - Train Loss: 74.4697, Test Loss: 8.4593


Epoch 2 - Training: 100%|██████████| 3913/3913 [03:45<00:00, 17.37it/s]


Model checkpoint epoch 1 saved to mimic_muse_slice_val/mimic_-1_muse_-1_bias_True/best_model.pth.tar
Epoch [2/100] - Train Loss: 7.4588, Test Loss: 7.5772


...

Epoch [77/100] - Train Loss: 5.1036, Test Loss: 5.4898


Epoch 78 - Training: 100%|██████████| 3913/3913 [03:42<00:00, 17.57it/s]


Epoch [78/100] - Train Loss: 5.1056, Test Loss: 5.4885


Epoch 79 - Training: 100%|██████████| 3913/3913 [03:44<00:00, 17.46it/s]


Epoch [79/100] - Train Loss: 5.1049, Test Loss: 5.5334
Stop early at epoch 79
Model checkpoint epoch 78 saved to mimic_muse_slice_val/mimic_-1_muse_-1_bias_True/checkpoint_epoch79.pth.tar
Training Complete!
Mimic
Mean Absolute Error (MAE): 4.8778
Mean Squared Error (MSE): 89.7819
Root Mean Squared Error (MSE): 9.4753
Muse biased
Mean Absolute Error (MAE): 16.8420
Mean Squared Error (MSE): 501.7653
Root Mean Squared Error (MSE): 22.4001
Muse unbiased
Mean Absolute Error (MAE): 10.3799
Mean Squared Error (MSE): 329.1381
Root Mean Squared Error (MSE): 18.1422
QTcMS auto
Mean Absolute Error (MAE): 18.9292
Mean Squared Error (MSE): 757.3018
Root Mean Squared Error (MSE): 27.5191
QTcMS expert
Mean Absolute Error (MAE): 14.3140
Mean Squared Error (MSE): 603.1612
Root Mean Squared Error (MSE): 24.5593
PTB
Mean Absolute Error (MAE): 24.9191
Mean Squared Error (MSE): 1333.2594
Root Mean Squared Error (MSE): 36.5138
ECGRDVQ
Mean Absolute Error (MAE): 8.4351
Mean Squared Error (MSE): 166.2766
Root